In [ ]:
import metpy
import cartopy.crs as ccrs
import hvplot.xarray
import matplotlib.pyplot as plt
import fsspec
import xarray as xr

Use metpy's `parse_cf()` to generate projection information for future plotting

In [ ]:
fs2 = fsspec.filesystem('reference', fo="./example_jsons/combined.json", remote_protocol='s3', remote_options=dict(anon=True), skip_instance_cache=True)
# fs2 = fsspec.filesystem('reference', fo="./combined.json", remote_protocol='s3', remote_options=dict(anon=True), skip_instance_cache=True)
ds = xr.open_dataset(fs2.get_mapper(""), engine='zarr').metpy.parse_cf()
ds

Use metpy to calculate lat/lon based on the GOES projection grid, and rename time dimension (for better plotting with hvplot)

In [ ]:
ds = ds.metpy.assign_latitude_longitude()
ds['t'].attrs['long_name'] = 'Time'

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
def get_xy_from_latlon(ds, lats, lons):
    import numpy as np
    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.latitude.data
    lon = ds.longitude.data
    
    x = ds.x.data
    y = ds.y.data
    
    x,y = np.meshgrid(x,y)
    
    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)] 
    
    return min(x), max(x), min(y), max(y)

In [ ]:
x1, x2, y1, y2 = get_xy_from_latlon(ds, (20,60), (-100, -50))
print(x1, x2, y1, y2)

In [ ]:
ds_subset = ds.sel(x=slice(x1, x2), y=slice(y2, y1))

In [ ]:
projection = ccrs.Orthographic(-75, 30)
transform = ds_subset.SST.metpy.cartopy_crs

fig = plt.figure()
ax = plt.subplot(projection=projection)
ax.pcolormesh(ds_subset.x, ds_subset.y, ds_subset.SST.isel(t=0), transform=transform)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)